<!--NOTEBOOK_INFORMATION-->
<img id="r-1060983" data-claire-element-id="1061343" src="http://www.siteduzero.com/favicon.ico" alt="Image utilisateur">
    <p>
        **<font color='#D2691E'size="6">Projet n°7 : Indexations automatiques d'images (4/9)</font>**.
    </p>
    <p>
        Ce notebook contient les étapes de construction du modèle "Bag of Visual Words" associé aux features SIFT des images du projet n°7.
    </p>

<p>
    <center>
        **<font color='	#D2691E'size="6">ROADMAP</font>**
    </center>
</p>
<img align="left" style="padding-right:10px;" src="./images/part_4.jpg">

<p>
    <center>
        **<font color='	#D2691E'size="6">PLAN</font>**
    </center>
</p>

<p>
        **<font color='#D2691E'size="4">0) Libraries, functions and datasources import</font>**
</p>
<p>
        **<font color='#D2691E'size="4">I) Clustering in a train data sample</font>**
</p>
<p>
        **<font color='#D2691E'size="4">I.1) Random data sampling</font>**
</p>
<p>
        **<font color='#D2691E'size="4">I.2) Dimension reduction with PCA</font>**
</p>
<p>
        **<font color='#D2691E'size="4">I.3) Clustering on the reduced sampled data</font>**
</p>
<p>
        **<font color='#D2691E'size="4">II.1) Dimension reduction with PCA</font>**
</p>
<p>
        **<font color='#D2691E'size="4">II.2) Clustering on the reduced data</font>**
</p>

<p>
        **<font color='#D2691E'size="4">0) Libraries, functions and datasources import</font>**
</p>

In [1]:
import pickle 
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import cv2
from sklearn import metrics, preprocessing 
from sklearn.cluster import AgglomerativeClustering,KMeans,Birch
from sklearn.decomposition import PCA
from sklearn.preprocessing  import StandardScaler
import random

In [2]:
from context import datasources_path, pickles_path, temp_files_path

In [3]:
from functions import values_to_col

In [4]:
CLUSTER_EXECUTION = 'RELOAD'
DATA_SCALER_AND_PCA = 'RELOAD'
N_RACES = pickle.load(open(pickles_path+"N_FILTERED_RACES_last_execution.p", "rb" ))
#N_RACES = 20

In [5]:
df_train = pickle.load(open(pickles_path+"df_train_%s_races.p"%N_RACES, "rb" ))
#40 races, avant egalisation histogramme : 5 357 663 lignes
#20 races, après egalisation histogramme : 4 134 228 lignes
#10 races, après egalisation histogramme : 2 172 070 lignes

In [6]:
df_test = pickle.load(open(pickles_path+"df_test_%s_races.p"%N_RACES, "rb" ))

In [7]:
dict_pictures_features_caracs = pickle.load(open(pickles_path+"dict_pictures_features_caracs_%s_races.p"%N_RACES, "rb" ))
dict_pictures_features_caracs[0]

{0: (12.05987548828125, 479.565185546875),
 1: (12.464170455932617, 287.9638671875),
 2: (12.781826972961426, 276.90771484375),
 3: (15.353494644165039, 487.3764343261719),
 4: (28.113872528076172, 241.62953186035156),
 5: (29.554725646972656, 374.8094482421875),
 6: (34.30908966064453, 114.2025375366211),
 7: (35.118709564208984, 471.7928466796875),
 8: (35.65925216674805, 153.85809326171875),
 9: (35.65925216674805, 153.85809326171875),
 10: (39.508262634277344, 184.56199645996094),
 11: (40.56473922729492, 152.76016235351562),
 12: (41.00375747680664, 158.52392578125),
 13: (41.00375747680664, 158.52392578125),
 14: (41.498252868652344, 78.64695739746094),
 15: (42.98707962036133, 107.04254913330078),
 16: (43.62761306762695, 119.42401885986328),
 17: (43.876399993896484, 263.8782653808594),
 18: (45.26868438720703, 490.8818664550781),
 19: (47.523170471191406, 88.49840545654297),
 20: (47.523170471191406, 88.49840545654297),
 21: (47.62620544433594, 97.39269256591797),
 22: (49.186

<p>
        **<font color='#D2691E'size="4">I) Clustering in a train data sample</font>**
</p>

<p>
        **<font color='#D2691E'size="4">I.1) Random data sampling</font>**
</p>

In [8]:
#Clustering
#In order to find clusters within the data, we are going to sample the "df_train" dataset with 10% of randomly selected features:

In [9]:
n_total_features = len(df_train)
L_indexes = list(range(0,n_total_features))
L_indexes

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [10]:
cluster_train_thresold = int(n_total_features*0.1)
cluster_train_thresold

217207

In [11]:
L_cluster_indexes = random.sample(L_indexes, cluster_train_thresold)
L_cluster_indexes = np.sort(L_cluster_indexes)
L_cluster_indexes

array([      2,       4,      33, ..., 2172045, 2172053, 2172059])

In [12]:
df_train_sample = df_train.copy()
df_train_sample = df_train_sample.loc[L_cluster_indexes]
df_train_sample

,B_value,G_value,R_value,feature_dominant_color,feature_id,picture_id,picture_name,race,0,1,...,127,feature_dominant_color_balanced,feature_dominant_color_blue,feature_dominant_color_gb,feature_dominant_color_green,feature_dominant_color_rb,feature_dominant_color_red,feature_dominant_color_rg,created_columns,dict_mapping
2,135,122,120,balanced,2,0,beagle_10108.jpg,beagle,11.0,2.0,...,106.0,1,0,0,0,0,0,0,"[feature_dominant_color_balanced, feature_domi...",{'feature_dominant_color': ['feature_dominant_...
4,148,138,138,balanced,4,0,beagle_10108.jpg,beagle,34.0,7.0,...,0.0,1,0,0,0,0,0,0,"[feature_dominant_color_balanced, feature_domi...",{'feature_dominant_color': ['feature_dominant_...
33,28,45,78,red,33,0,beagle_10108.jpg,beagle,0.0,0.0,...,1.0,0,0,0,0,0,1,0,"[feature_dominant_color_balanced, feature_domi...",{'feature_dominant_color': ['feature_dominant_...
53,33,47,83,red,53,0,beagle_10108.jpg,beagle,0.0,3.0,...,0.0,0,0,0,0,0,1,0,"[feature_dominant_color_balanced, feature_domi...",{'feature_dominant_color': ['feature_dominant_...
71,22,23,27,red,71,0,beagle_10108.jpg,beagle,28.0,53.0,...,0.0,0,0,0,0,0,1,0,"[feature_dominant_color_balanced, feature_domi...",{'feature_dominant_color': ['feature_dominant_...
85,66,87,118,red,85,0,beagle_10108.jpg,beagle,19.0,1.0,...,23.0,0,0,0,0,0,1,0,"[feature_dominant_color_balanced, feature_domi...",{'feature_dominant_color': ['feature_dominant_...
88,181,168,160,balanced,88,0,beagle_10108.jpg,beagle,81.0,35.0,...,1.0,1,0,0,0,0,0,0,"[feature_dominant_color_balanced, feature_domi...",{'feature_dominant_color': ['feature_dominant_...
98,189,176,162,balanced,98,0,beagle_10108.jpg,beagle,66.0,37.0,...,0.0,1,0,0,0,0,0,0,"[feature_dominant_color_balanced, feature_domi...",{'feature_dominant_color': ['feature_dominant_...
103,35,42,57,red,103,0,beagle_10108.jpg,beagle,14.0,38.0,...,1.0,0,0,0,0,0,1,0,"[feature_dominant_color_balanced, feature_domi...",{'feature_dominant_color': ['feature_dominant_...
158,108,121,137,red,158,0,beagle_10108.jpg,beagle,22.0,4.0,...,0.0,0,0,0,0,0,1,0,"[feature_dominant_color_balanced, feature_domi...",{'feature_dominant_color': ['feature_dominant_...


<p>
        **<font color='#D2691E'size="4">I.2) Dimension reduction with PCA</font>**
</p>

In [13]:
L_interest_features = ['R_value','G_value','B_value']
L_interest_features+= df_train['created_columns'][0].copy()
L_interest_features+=list(range(0,128))
L_interest_features

['R_value',
 'G_value',
 'B_value',
 'feature_dominant_color_balanced',
 'feature_dominant_color_blue',
 'feature_dominant_color_gb',
 'feature_dominant_color_green',
 'feature_dominant_color_rb',
 'feature_dominant_color_red',
 'feature_dominant_color_rg',
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127]

In [14]:
X_sample = df_train_sample[L_interest_features].values
if DATA_SCALER_AND_PCA == 'RELOAD':  
    std_scale = StandardScaler().fit(X_sample)
    pickle.dump(std_scale, open(pickles_path+"data_scaler.p", "wb"))

if DATA_SCALER_AND_PCA == 'LOAD':
    std_scale = pickle.load(open(pickles_path+"data_scaler.p", "rb" ))

X_scaled_sample = std_scale.transform(X_sample)

c:\python\python36\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [15]:
if DATA_SCALER_AND_PCA == 'RELOAD':  
    pca = PCA(n_components=80)
    pca.fit(X_scaled_sample)
    pickle.dump(pca, open(pickles_path+"pca_fitted.p", "wb"))

if DATA_SCALER_AND_PCA == 'LOAD':
    pca = pickle.load(open(pickles_path+"pca_fitted.p", "rb" ))

print(pca.explained_variance_ratio_.cumsum())
X_PCA_sample = pca.transform(X_scaled_sample)

[0.07486296 0.13619084 0.18363129 0.23006669 0.27389597 0.31285184
 0.34357981 0.37401289 0.40400216 0.42827622 0.45092436 0.47235714
 0.49209084 0.50997683 0.52690604 0.54370906 0.55980393 0.57471688
 0.589132   0.60304201 0.61693324 0.62979609 0.64173938 0.65278008
 0.66328861 0.67267785 0.68189125 0.69102591 0.69997296 0.7084709
 0.71636226 0.72424706 0.73179627 0.73923908 0.7465074  0.75375472
 0.76085691 0.76786679 0.77481533 0.78171892 0.78818009 0.79451114
 0.80081095 0.80707947 0.81309622 0.81905594 0.82452486 0.82996264
 0.83533896 0.84051544 0.84550583 0.85039937 0.85515328 0.85976131
 0.86409827 0.86831515 0.87232346 0.87624596 0.88008013 0.88381132
 0.88749157 0.89112568 0.89461516 0.89800883 0.9011424  0.90427382
 0.90729316 0.91030894 0.91314706 0.91592982 0.91860893 0.92127053
 0.92388959 0.92644737 0.92891483 0.93120497 0.93347539 0.93572557
 0.93793262 0.9401153 ]


<p>
        **<font color='#D2691E'size="4">I.3) Clustering on the reduced sampled data</font>**
</p>

In [16]:
if CLUSTER_EXECUTION == 'RELOAD':
    
    X_to_fit = X_PCA_sample

    %pylab inline
    #On implémente différents clustering sur plusieurs tailles de clusters :

    dict_clustering_models = {'GaussianMixture':{},'KMeans':{},'Birch':{}}
    dict_labels = {'GaussianMixture':{},'KMeans':{},'Birch':{}}
    dict_clusters_centers = {'GaussianMixture':{},'KMeans':{},'Birch':{}}
    
    for num_clusters in [50,100,200,300,500]:
        start_time = time.time()
        print("itération sur %s clusters"%num_clusters)
        
        #We implement a KMeans clustering object :
        init_fitting_time = time.time()
        KM = KMeans(n_clusters=num_clusters, random_state =42, init='k-means++')
        KM = KM.fit(X_to_fit)
        labels_KM = KM.labels_
        cluster_centers_KM = KM.cluster_centers_
        print("KM fitting_time : --- %s seconds ---" % (time.time() - init_fitting_time))
        
        dict_clustering_models['KMeans'][num_clusters] = KM
        dict_labels['KMeans'][num_clusters] = labels_KM
        dict_clusters_centers['KMeans'][num_clusters] = cluster_centers_KM
        
        
        pickle.dump(dict_clustering_models,open(pickles_path+"dict_clustering_models_%s_%s_races.p"%(num_clusters, N_RACES), "wb"))
        pickle.dump(dict_labels,open(pickles_path+"dict_labels_%s_%s_races.p"%(num_clusters, N_RACES), "wb"))
        pickle.dump(dict_clusters_centers,open(pickles_path+"dict_cluster_centers_%s_%s_races.p"%(num_clusters, N_RACES), "wb"))

    print("--- %s seconds ---" % (time.time() - start_time))

Populating the interactive namespace from numpy and matplotlib
itération sur 50 clusters


c:\python\python36\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


KM fitting_time : --- 947.3254897594452 seconds ---
itération sur 100 clusters
KM fitting_time : --- 1732.3375833034515 seconds ---
itération sur 200 clusters
KM fitting_time : --- 3325.4132068157196 seconds ---
itération sur 300 clusters
KM fitting_time : --- 4143.526103973389 seconds ---
itération sur 500 clusters
KM fitting_time : --- 6476.696573257446 seconds ---
--- 6477.219176054001 seconds ---


<p>
        **<font color='#D2691E'size="4">II) Clustering in the full data perimeter</font>**
</p>

<p>
        **<font color='#D2691E'size="4">II.1) Dimension reduction with PCA</font>**
</p>

In [17]:
from sklearn.preprocessing import LabelEncoder

In [18]:
L_all_races = list(np.unique(df_train['race']))
labelencoder = LabelEncoder()
L_encoded_races = labelencoder.fit_transform(L_all_races)
pickle.dump(labelencoder,open(pickles_path+"labelencoder_%s_races.p"%N_RACES, "wb"))

In [19]:
print('Applying the PCA on the train data ...')
X_train = df_train[L_interest_features].values
X_scaled_train  = std_scale.transform(X_train)
X_PCA_train  = pca.transform(X_scaled_train)
    
print('Applying the PCA on the test data ...')
X_test = df_test[L_interest_features].values
X_scaled_test  = std_scale.transform(X_test)
X_PCA_test  = pca.transform(X_scaled_test)

Applying the PCA on the train data ...


c:\python\python36\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Applying the PCA on the test data ...


<p>
        **<font color='#D2691E'size="4">II.2) Clustering on the reduced data</font>**
</p>

In [20]:
for num_clusters in [50,100,200,300,500]:
    print()
    print('##################')
    print('num_clusters = %s'%num_clusters)
    N_FINAL_CLUSTERS = num_clusters
    FINAL_CLUSTERING = 'KMeans'
    
    dict_labels = pickle.load(open(pickles_path+"dict_labels_%s_%s_races.p"%(N_FINAL_CLUSTERS,N_RACES), "rb" ))
    labels_KM = dict_labels['KMeans'][N_FINAL_CLUSTERS]
    cluster_centers_KM = pickle.load(open(pickles_path+"dict_cluster_centers_%s_%s_races.p"%(N_FINAL_CLUSTERS,N_RACES), "rb" ))
    
    dict_clustering_models = pickle.load(open(pickles_path+"dict_clustering_models_%s_%s_races.p"%(N_FINAL_CLUSTERS,N_RACES), "rb" ))
    
    df_train_sample['cluster'] = dict_labels[FINAL_CLUSTERING][N_FINAL_CLUSTERS]
    
    #Applying the clustering to the full dataframes :
    
    clustering_model = dict_clustering_models[FINAL_CLUSTERING][N_FINAL_CLUSTERS]
    
    print('Predicting the clusters (train data) ... ')
    L_train_clusters = clustering_model.predict(X_PCA_train)
    print('Setting clusters labels to the train dataframe ...')
    df_train['cluster'] = L_train_clusters
    df_train['cluster'] = df_train['cluster'].astype(str)
    
    print('Predicting the clusters (test data) ... ')
    L_test_clusters = clustering_model.predict(X_PCA_test)
    print('Setting clusters labels to the test dataframe ...')
    df_test['cluster'] = L_test_clusters
    df_test['cluster'] = df_test['cluster'].astype(str)
    
    L_all_races = list(np.unique(df_train['race']))
    
    print('Creating the bag of visual words for each races ...')
    for race in L_all_races:
        print(race)
        sub_df_train = df_train[['picture_name','cluster']][df_train['race'] == race].copy()
        sub_df_test = df_test[['picture_name','cluster']][df_test['race'] == race].copy()

        sub_df_train.index = range(0,len(sub_df_train))
        sub_df_test.index = range(0,len(sub_df_test))

        sub_df_train = values_to_col(sub_df_train,['cluster'],True)
        sub_df_test = values_to_col(sub_df_test,['cluster'],True)

        focus_columns_train = sub_df_train['created_columns'][0]
        focus_columns_train+=['picture_name']

        focus_columns_test = sub_df_test['created_columns'][0]
        focus_columns_test+=['picture_name']

        sub_df_train = sub_df_train[focus_columns_train].groupby('picture_name').sum()
        sub_df_test = sub_df_test[focus_columns_test].groupby('picture_name').sum()

        sub_df_train_name = 'bag_of_visual_words_train_%s_races_%s_clusters'%(race,num_clusters)
        pickle.dump(sub_df_train,open(temp_files_path+sub_df_train_name+".p", "wb"))

        sub_df_test_name = 'bag_of_visual_words_test_%s_races_%s_clusters'%(race,num_clusters)
        pickle.dump(sub_df_test,open(temp_files_path+sub_df_test_name+".p", "wb"))
    
    focus_columns = ['picture_name']
    focus_columns+= ["cluster_%s"%i for i in range(0,N_FINAL_CLUSTERS)]
    
    df_model_train = pd.DataFrame(columns = focus_columns)
    df_model_test = df_model_train.copy()
    
    for race in L_all_races:
        sub_df_train_name = 'bag_of_visual_words_train_%s_races_%s_clusters'%(race,num_clusters)
        sub_df_test_name = 'bag_of_visual_words_test_%s_races_%s_clusters'%(race,num_clusters)

        sub_df_train = pickle.load(open(temp_files_path+sub_df_train_name+".p", "rb" ))
        sub_df_test = pickle.load(open(temp_files_path+sub_df_test_name+".p", "rb" ))

        sub_df_train['picture_name'] = sub_df_train.index
        sub_df_test['picture_name'] = sub_df_test.index

        df_model_train = df_model_train.append(sub_df_train)
        df_model_test = df_model_test.append(sub_df_test)

    df_model_train.fillna(0,inplace=True)
    df_model_test.fillna(0,inplace=True)

    df_model_train = df_model_train[focus_columns]
    df_model_test = df_model_test[focus_columns]
    
    
   #Now we have this configuration we need to normalize the picture/cluster distribution for each train and test dataframes : 
    
    orig_picture_names_train = df_model_train['picture_name'].copy()
    orig_picture_names_test = df_model_test['picture_name'].copy()
    
    focus_columns_with_picture_name = focus_columns.copy()
    focus_columns.remove('picture_name')
    print('Normalizing the bag of visual words (train data) ...')
    start_time = time.time()
    for row in df_model_train.index:
        row_sum = np.sum(df_model_train.loc[row,focus_columns])
        df_model_train.loc[row,focus_columns] = df_model_train.loc[row,focus_columns]/row_sum

    print("--- %s seconds ---" % (time.time() - start_time))
    df_model_train['picture_name'] = orig_picture_names_train
    df_model_train = df_model_train[focus_columns_with_picture_name]
    
    print('Normalizing the bag of visual words (test data) ...')
    start_time = time.time()
    for row in df_model_test.index:
        row_sum = np.sum(df_model_test.loc[row,focus_columns])
        df_model_test.loc[row,focus_columns] = df_model_test.loc[row,focus_columns]/row_sum

    print("--- %s seconds ---" % (time.time() - start_time))
    df_model_test['picture_name'] = orig_picture_names_test
    df_model_test = df_model_test[focus_columns_with_picture_name]
    
    
    df_race_picture_train = df_train[['race','picture_name']].copy()
    df_race_picture_train.drop_duplicates(subset=['race','picture_name'], inplace=True)
    df_race_picture_test = df_test[['race','picture_name']].copy()
    df_race_picture_test.drop_duplicates(subset=['race','picture_name'], inplace=True)
    
    df_model_train = pd.merge(df_model_train, df_race_picture_train, how='left', on='picture_name')
    df_model_test = pd.merge(df_model_test, df_race_picture_test, how='left', on='picture_name')
    
    print('Encoding the "race" column ...')
    df_model_train['encoded_race'] = df_model_train['race'].apply(lambda x : labelencoder.transform([x])[0])
    df_model_test['encoded_race'] = df_model_test['race'].apply(lambda x : labelencoder.transform([x])[0])
    
    print('Dumping the train model ...')
    pickle.dump(df_model_train,open(pickles_path+"df_model_train_with_%s_clusters_%s_races.p"%(N_FINAL_CLUSTERS,N_RACES), "wb"))
    print('Dumping the test model ...')
    pickle.dump(df_model_test,open(pickles_path+"df_model_test_with_%s_clusters_%s_races.p"%(N_FINAL_CLUSTERS,N_RACES), "wb"))


##################
num_clusters = 50
Predicting the clusters (train data) ... 
Setting clusters labels to the train dataframe ...
Predicting the clusters (test data) ... 
Setting clusters labels to the test dataframe ...
Creating the bag of visual words for each races ...
beagle
bernese_mountain_dog
dhole
english_setter
japanese_spaniel
kelpie
labrador_retriever
rottweiler
siberian_husky
west_highland_white_terrier
Normalizing the bag of visual words (train data) ...
--- 51.89432239532471 seconds ---
Normalizing the bag of visual words (test data) ...
--- 8.292830228805542 seconds ---
Encoding the "race" column ...
Dumping the train model ...
Dumping the test model ...

##################
num_clusters = 100
Predicting the clusters (train data) ... 
Setting clusters labels to the train dataframe ...
Predicting the clusters (test data) ... 
Setting clusters labels to the test dataframe ...
Creating the bag of visual words for each races ...
beagle
bernese_mountain_dog
dhole
english_sett